In [38]:
from graph_tool.all import *
from numpy.random import randint
from numpy.random import random
from numpy.random import *
from IPython.display import Image #import to displaz images
from matplotlib import *
from gi.repository import Gtk, Gdk, GdkPixbuf, GObject, GLib
import sys, os, os.path
import time
import numpy as np

#things to be done, figure out all the movement and movement in between states
#add different infection probabilities for vaccination and age group
#if person is vaccinated and not vaccinated


In [30]:
g = Graph(directed=False)

#definition of vertex properties
S = [1, 1, 1, 1]           # White color

I = [0, 0, 0, 1]           # Black color

R = [0.5, 0.5, 0.5, 1.]    # Grey color (will not actually be drawn)

V = [0, 0, 1, 1]           # Blue color

D = [0.8, 0, 0, 0.6]       #DEATH (red)

#all definitions of vertice edges
state = g.new_vertex_property("vector<double>")
age=g.new_vertex_property("int")
vac=g.new_vertex_property("bool")
removed = g.new_vertex_property("bool")
newly_infected = g.new_vertex_property("bool")

agegroups=20
agegrouplists = [[] for i in range(1, agegroups+1)]

#creation of age buckets
for v in g.vertices():
    i=randint(1,age_groups)
    vprop_age[v] = i
    

    
#probabilities
x = 0  # spontaneous outbreak probability

r = 0.1      # I->R probability

s = 0    # R->S probability

#death probability list for different age groups
drlist = [0,0,0,0,0,0,0,0,0,0,0,0,.10,.20,.30,.40,.50,.60,.90,1]
#infection probability for different age groups
inflist = [0,0,0,0,0,0,0,0,0,0,0,0,.10,.20,.30,.40,.50,.60,.90,1]

In [69]:
for i in range(0,agegroups):
    print(i+1,': ',len(agegrouplists[i]))

1 :  0
2 :  24
3 :  23
4 :  23
5 :  28
6 :  29
7 :  29
8 :  25
9 :  35
10 :  42
11 :  48
12 :  41
13 :  34
14 :  39
15 :  38
16 :  41
17 :  35
18 :  24
19 :  23
20 :  14


In [70]:
print(agegrouplists)

[[], [<Vertex object with index '12' at 0x7f70b68a0b30>, <Vertex object with index '20' at 0x7f70b68a0ba0>, <Vertex object with index '50' at 0x7f70b6898cf0>, <Vertex object with index '77' at 0x7f70b689cac0>, <Vertex object with index '16' at 0x7f70b5482430>, <Vertex object with index '107' at 0x7f70b4fe7c80>, <Vertex object with index '152' at 0x7f70b4ff70b0>, <Vertex object with index '18' at 0x7f70b5482510>, <Vertex object with index '76' at 0x7f70b5437b30>, <Vertex object with index '222' at 0x7f70b543ac10>, <Vertex object with index '163' at 0x7f70b5439200>, <Vertex object with index '257' at 0x7f70b543cba0>, <Vertex object with index '212' at 0x7f70b4da80b0>, <Vertex object with index '158' at 0x7f70b543f890>, <Vertex object with index '292' at 0x7f70b4dab430>, <Vertex object with index '15' at 0x7f70b543d3c0>, <Vertex object with index '308' at 0x7f70b4f16200>, <Vertex object with index '51' at 0x7f70b4d8af90>, <Vertex object with index '326' at 0x7f70b4f169e0>, <Vertex object 

In [53]:
ages = np.genfromtxt('Age_Dists/JapanDist.csv', delimiter=',')[:,1]
print(ages)

[4. 4. 4. 4. 5. 5. 5. 6. 7. 8. 7. 6. 6. 6. 7. 6. 4. 3. 2. 0.]


In [67]:
def add_ages(g,distribution):
    path1 = 'Age_Dists/'
    path2 = 'Dist.csv'
    #generate array containing amount of people per age group
    agedistribution = np.genfromtxt(path1+distribution+path2, delimiter=',')[:,1]
    #visit vertices in random order
    
    vs = list(g.vertices())
    shuffle(vs)
    #we will fill all age groups one by one
    a = 1 #current agegroup
    counter = 0 #counter for how many people have been placed in current group
    
    for v in vs:
        if(a > 20):
            #safety measure
            return
        if counter < agedistribution[a-1]:
            #if current age group hasn't gotten enough people assigned, add another, move on to next vertix
            age[v] = a
            agegrouplists[a].append(v)
            counter = counter + 1
        else:
            #current age group is full, move to next age group with at least one person
            a = a + 1
            while(agedistribution[a-1] == 0):
                if(a < 20):
                    a = a + 1
                else:
                    #in case number for last age group is 0
                    return
            age[v] = a
            agegrouplists[a].append(v)
            counter = 1

In [68]:
#Creation of graph
#insert random vertices (nodes)
g.add_vertex(100)

# insert some random links
for s,t in zip(randint(0, 100, 100), randint(0, 100, 100)):
    g.add_edge(g.vertex(s), g.vertex(t))
# layout positions
pos = sfdp_layout(g)


#makes all nodes suceptible
for i in g.vertices():
    v = g.vertex(i) 
    state[v] = S

#this line is to randomlyinitialize one vertex as infected
state[randint(0,100)]= I

'''#gives all nodes an age and then adds nodes to an age group list
for i in g.vertices():
    v = g.vertex(i)
    a =randint(1,agegroups)
    age[v] = a
    agegrouplists[a].append(v)
'''
add_ages(g,'Japan')

#current placeholder for adding vaccination properties
if 1 == 2:
    for i in g.vertices():
        v = g.vertex(i)
        vac[v] = False

        
#Displaying how the graph will be initialized
graph_draw(g, vertex_text=g.vertex_index, output="two-nodes.png")
fig = Image(filename=('two-nodes.png'))
fig


# If True, the frames will be dumped to disk as images.
# CAN SOMONE FIGURE OUT WHAT THIS DOES???????????????????????????????

offscreen = sys.argv[1] == "offscreen" if len(sys.argv) > 1 else False

max_count = 500

if offscreen and not os.path.exists("./frames"):

    os.mkdir("./frames")
#?????????????????????????????????????????????????????????????????????


# This creates a GTK+ window with the initial graph layout,basically creates the pop up window

if not offscreen:

    win = GraphWindow(g, pos, geometry=(500, 400),

                      edge_color=[0.6, 0.6, 0.6, 1],

                      vertex_fill_color=state,

                      vertex_halo=newly_infected,

                      vertex_halo_color=[0.8, 0, 0, 0.6])

else:

    count = 0

    win = Gtk.OffscreenWindow()

    win.set_default_size(500, 400)

    win.graph = GraphWidget(g, pos,

                            edge_color=[0.6, 0.6, 0.6, 1],

                            vertex_fill_color=state,

                            vertex_halo=newly_infected,

                            vertex_halo_color=[0.8, 0, 0, 0.6])

    win.add(win.graph)

In [11]:
drlist[age[10]]

0

In [5]:
def update_state():

    newly_infected.a = False

    removed.a = False


    # visit the nodes in random order

    vs = list(g.vertices())

    shuffle(vs)

    for v in vs:

        if state[v] == I:

            if random() < r: #r: recovery rate

                state[v] = R
                
            if random() < drlist[age[v]]:
            
                state[v] = D
                g.clear_vertex(v)

        elif state[v] == S:

            if random() < x: #x: spontaneous infection rate -> do we reallz need this not really?

                state[v] = I

            else:

                ns = list(v.out_neighbors())

                if len(ns) > 0:

                    w = ns[randint(0, len(ns))]  # choose a random neighbor

                    if state[w] == I:
                        
                        #you shouldnt automatically be infected just because your neighbor is, there should still be an infection rate

                        state[v] = I

                        newly_infected[v] = True
            # add transition to new state V - vaccinated

       #elif random() < s: #what is this supposed to be?

       #    state[v] = S

        if state[v] == R:

            removed[v] = True
       

    # Filter out the recovered vertices

    g.set_vertex_filter(removed, inverted=True)


    # The following will force the re-drawing of the graph, and issue a

    # re-drawing of the GTK window.

    win.graph.regenerate_surface()
    time.sleep(0.5)
    win.graph.queue_draw()


    # if doing an offscreen animation, dump frame to disk

    if offscreen:

        global count

        pixbuf = win.get_pixbuf()

        pixbuf.savev(r'./frames/sirs%06d.png' % count, 'png', [], [])

        if count > max_count:

            sys.exit(0)

        count += 1


    # We need to return True so that the main loop will call this function more

    # than once.

    return True


In [21]:
cid = GLib.idle_add(update_state)


# We will give the user the ability to stop the program by closing the window.

win.connect("delete_event", Gtk.main_quit)


# Actually show the window, and start the main loop.

win.show_all()

Gtk.main()

In [11]:
g.list_properties()

In [8]:
g.save("my_graph.xml.gz")
g2 = load_graph("my_graph.xml.gz")